# Análisis de sentimientos en español

##### Jean Pierre Londoño González
##### Mini-proyecto de procesamiento de texto utilizando técnicas clásicas
##### 16AGO2025

### 1. Importación de librerias y carga de modelos

Inicio importando las librerías necesarias para el procesamiento de lenguaje natural, la traducción automática y la evaluación del modelo. Esto incluye spaCy para el análisis gramatical, deep_translator para traducir texto de español a inglés, y nltk junto con SentiWordNet para calcular puntajes de sentimiento. También se importan librerías de evaluación como sklearn.metrics para medir la precisión y generar reportes de clasificación.

In [129]:
# Manejo de paquetes y recursos
import pkg_resources  # Permite verificar versiones y dependencias de paquetes instalados

# Procesamiento de lenguaje natural
import spacy  # Biblioteca para NLP avanzada (tokenización, POS tagging, etc.)
import pandas as pd  # Manipulación y análisis de datos en estructuras tipo DataFrame
import warnings  # Control y filtrado de mensajes de advertencia
from nltk.corpus import sentiwordnet as swn  # Léxico de sentimientos basado en WordNet
from nltk.corpus import wordnet as wn  # Diccionario semántico de palabras en inglés
from deep_translator import GoogleTranslator  # Traducción de texto usando Google Translate
import nltk  # Biblioteca general de NLP para tokenización, stemming, etc.

#  Métricas de evaluación
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report  # Métricas de evaluación

# Desactiva advertencias para mantener limpia la salida
warnings.filterwarnings('ignore')

A continuación, se realiza la descarga de los recursos necesarios de NLTK para el análisis semántico y el cálculo de puntajes de sentimiento. Esto incluye wordnet para acceder a la base de datos léxica de WordNet, sentiwordnet para obtener las puntuaciones positivas y negativas de cada término, y averaged_perceptron_tagger para el etiquetado gramatical. Estos modelos son indispensables para que el sistema pueda identificar correctamente la categoría gramatical de cada palabra y asignarle un puntaje de sentimiento adecuado.

In [130]:
# Descarga de recursos necesarios de NLTK
nltk.download('sentiwordnet')  # Léxico de sentimientos basado en WordNet
nltk.download('wordnet')       # Base de datos léxica para relaciones semánticas
nltk.download('omw-1.4')       # Open Multilingual Wordnet (traducciones y sinónimos)

[nltk_data] Downloading package sentiwordnet to
[nltk_data]     C:\Users\jepil\AppData\Roaming\nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jepil\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\jepil\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

Se carga el modelo es_core_news_lg de spaCy, que incluye vectores de palabras preentrenados y capacidades avanzadas de análisis gramatical para el idioma español. Este modelo permite identificar tokens, realizar lematización, determinar la categoría gramatical (POS tagging) y extraer dependencias sintácticas, lo cual es esencial para el procesamiento y análisis semántico del texto en español.

In [131]:
# Cargar el modelo
nlp = spacy.load('es_core_news_lg')

Se crea un dataset simulado compuesto por textos en español junto con sus respectivas etiquetas de clasificación de sentimiento: positivo y negativo. Este conjunto de datos sirve como base para realizar las pruebas y validaciones del modelo, permitiendo evaluar su capacidad de identificar y clasificar correctamente las emociones expresadas en los textos.

In [132]:
training_data = [
    # ==================== POSITIVO ====================
    ("Me encantó la película, la historia fue emocionante y los actores brillaron.", "positivo"),
    ("La comida en ese lugar siempre me deja con una sonrisa, riquísima.", "positivo"),
    ("Gracias al equipo de soporte por resolver mi problema en menos de 10 minutos.", "positivo"),
    ("El nuevo diseño de la aplicación es muy intuitivo, me resulta facilísimo de usar.", "positivo"),
    ("Disfruté muchísimo la conferencia, aprendí más de lo que esperaba.", "positivo"),
    ("El paquete llegó antes de la fecha estimada, muy satisfecho.", "positivo"),
    ("Es un placer trabajar con un equipo tan profesional y amable.", "positivo"),
    ("El hotel tenía una vista espectacular al mar, volveré pronto.", "positivo"),
    ("Me encanta cómo huele este perfume, es simplemente perfecto.", "positivo"),
    ("Qué alegría recibir buenas noticias tan temprano en la mañana.", "positivo"),
    ("Agradezco mucho la atención personalizada, me hicieron sentir importante.", "positivo"),
    ("Este libro me atrapó desde la primera página, lectura recomendada.", "positivo"),
    ("La música en vivo fue una experiencia que no olvidaré jamás.", "positivo"),
    ("Todo el proceso de compra fue rápido y sin complicaciones.", "positivo"),
    ("Me sorprendió gratamente el sabor de la bebida, deliciosa.", "positivo"),
    ("El trato cordial y la rapidez hacen que siempre elija este lugar.", "positivo"),
    ("La experiencia en el parque temático fue divertida de principio a fin.", "positivo"),
    ("La presentación estuvo impecable, bien estructurada y clara.", "positivo"),
    ("Estoy muy feliz con mi nuevo trabajo, todo es como lo soñaba.", "positivo"),
    ("Recibir ese reconocimiento me motivó muchísimo a seguir mejorando.", "positivo"),
    ("La atención al cliente superó todas mis expectativas.", "positivo"),
    ("Me siento agradecido por la ayuda que me brindaron.", "positivo"),
    ("El nuevo menú tiene combinaciones deliciosas, ¡vale la pena probarlo!", "positivo"),
    ("La clase de hoy fue muy entretenida y bien explicada.", "positivo"),
    ("El viaje a la montaña fue relajante y lleno de paisajes hermosos.", "positivo"),
    ("El software funciona a la perfección, sin errores.", "positivo"),
    ("Me siento inspirado después de escuchar esa charla motivacional.", "positivo"),
    ("Las flores del jardín están más bonitas que nunca este año.", "positivo"),
    ("Amo cómo decoraron la tienda para la temporada navideña.", "positivo"),
    ("El servicio rápido y amable hace toda la diferencia.", "positivo"),
    ("El pastel que preparaste estaba espectacular, suave y delicioso.", "positivo"),
    ("La atención de los meseros fue impecable toda la noche.", "positivo"),
    ("Mi computadora nueva es rapidísima, no se traba para nada.", "positivo"),
    ("El envío llegó en perfectas condiciones, bien embalado.", "positivo"),
    ("El curso online está muy bien explicado y con ejemplos claros.", "positivo"),
    ("Las vistas desde el mirador son impresionantes.", "positivo"),
    ("El ambiente en la cafetería es acogedor y relajante.", "positivo"),
    ("El servicio técnico resolvió el problema en una sola llamada.", "positivo"),
    ("Las vacaciones en la playa fueron exactamente lo que necesitaba.", "positivo"),
    ("La exposición de arte fue inspiradora y bien organizada.", "positivo"),
    ("El café de esta mañana estaba perfecto, justo como me gusta.", "positivo"),
    ("El regalo que recibí fue muy especial y significativo.", "positivo"),
    ("Me encanta el aroma de este jabón artesanal.", "positivo"),
    ("El trato en la tienda fue muy cordial y profesional.", "positivo"),
    ("El juego nuevo tiene gráficos espectaculares y buena jugabilidad.", "positivo"),
    ("Me encantó cómo decoraron la sala para la reunión.", "positivo"),
    ("La charla motivacional me dejó lleno de energía.", "positivo"),
    ("La película me mantuvo al borde del asiento.", "positivo"),
    ("El servicio de streaming funciona sin interrupciones.", "positivo"),
    ("La biblioteca tiene una gran selección de libros interesantes.", "positivo"),
    ("La cena de anoche fue una de las mejores que he probado.", "positivo"),
    ("El taller fue práctico y fácil de seguir.", "positivo"),
    ("El trato que recibí fue cálido y humano.", "positivo"),
    ("Las clases de yoga me ayudaron a relajarme muchísimo.", "positivo"),
    ("Me alegra haber tomado esta decisión, fue la correcta.", "positivo"),
    ("El producto funciona tal como se describe, excelente compra.", "positivo"),
    ("La conferencia tuvo ponentes muy preparados.", "positivo"),
    ("El postre estaba delicioso y bien presentado.", "positivo"),
    ("Me impresionó la rapidez de la entrega.", "positivo"),
    ("El equipo de trabajo es muy colaborador.", "positivo"),
    ("Me encantó la excursión por la montaña.", "positivo"),
    ("El hotel tiene un spa increíble.", "positivo"),
    ("El helado artesanal tiene un sabor único.", "positivo"),
    ("El concierto estuvo lleno de energía.", "positivo"),
    ("La tienda ofrece productos de gran calidad.", "positivo"),
    ("La atención fue personalizada y amable.", "positivo"),
    ("La serie que vi anoche es adictiva.", "positivo"),
    ("El paquete llegó un día antes de lo esperado.", "positivo"),
    ("El producto es mejor de lo que imaginaba.", "positivo"),
    ("El nuevo menú es variado y delicioso.", "positivo"),
    ("El servicio postventa fue rápido y efectivo.", "positivo"),
    ("La vista desde la habitación era espectacular.", "positivo"),
    ("El ambiente era relajado y acogedor.", "positivo"),
    ("La música en el bar era excelente.", "positivo"),
    ("El taller superó mis expectativas.", "positivo"),
    ("El curso fue muy útil y claro.", "positivo"),
    ("La decoración era preciosa y elegante.", "positivo"),
    ("El café estaba recién hecho y delicioso.", "positivo"),
    ("La experiencia fue agradable y enriquecedora.", "positivo"),
    ("Me encanta visitar este lugar cada verano.", "positivo"),
    ("Las atenciones que recibí fueron excelentes.", "positivo"),
    ("El hotel cumplió todo lo prometido.", "positivo"),
    ("La organización del evento fue impecable.", "positivo"),
    ("Me alegró ver a tantos amigos reunidos.", "positivo"),
    ("La limpieza del lugar era impecable.", "positivo"),
    ("El chef preparó un plato espectacular.", "positivo"),
    ("El clima de hoy es perfecto para caminar.", "positivo"),
    ("El ambiente laboral es positivo y motivador.", "positivo"),
    ("El producto es resistente y duradero.", "positivo"),
    ("La atención fue rápida y eficiente.", "positivo"),
    ("El parque estaba limpio y bien cuidado.", "positivo"),
    ("El documental fue interesante y educativo.", "positivo"),
    ("Me encantó el detalle del regalo.", "positivo"),
    ("La comida estaba recién hecha y deliciosa.", "positivo"),
    ("El servicio fue atento y rápido.", "positivo"),
    ("La entrega fue puntual y sin problemas.", "positivo"),
    ("El show fue divertido y emocionante.", "positivo"),
    ("El curso me ayudó a mejorar mis habilidades.", "positivo"),
    ("La experiencia de compra fue excelente.", "positivo"),
    ("La ropa es de muy buena calidad.", "positivo"),
    # ==================== NEGATIVO ====================
    ("Odié la experiencia de atención al cliente, no resolvieron nada.", "negativo"),
    ("La página web se congela cada vez que intento pagar.", "negativo"),
    ("El ruido de la calle no me dejó dormir en toda la noche.", "negativo"),
    ("La comida estaba fría cuando llegó a la mesa.", "negativo"),
    ("El curso fue confuso y mal explicado.", "negativo"),
    ("El transporte público estuvo lleno y fue incómodo.", "negativo"),
    ("No recibí el reembolso que me prometieron.", "negativo"),
    ("La tienda online no tiene un buen sistema de búsqueda.", "negativo"),
    ("La película tuvo un final predecible y aburrido.", "negativo"),
    ("El aire acondicionado no funcionaba en el hotel.", "negativo"),
    ("El trato por parte del personal fue irrespetuoso.", "negativo"),
    ("El paquete llegó abierto y con cosas faltantes.", "negativo"),
    ("El sonido en el evento era pésimo.", "negativo"),
    ("La habitación tenía manchas en las sábanas.", "negativo"),
    ("El taxi me cobró más de lo indicado.", "negativo"),
    ("El servicio técnico nunca respondió a mis mensajes.", "negativo"),
    ("El nuevo parche del videojuego arruinó la jugabilidad.", "negativo"),
    ("Las instrucciones del producto no son claras.", "negativo"),
    ("El café estaba quemado y sabía amargo.", "negativo"),
    ("El internet se desconecta constantemente.", "negativo"),
    ("El restaurante no cumplió con mi reserva.", "negativo"),
    ("La presentación fue demasiado larga y aburrida.", "negativo"),
    ("El clima arruinó nuestras vacaciones.", "negativo"),
    ("El software tiene demasiados errores y fallos.", "negativo"),
    ("El empleado fue grosero y poco servicial.", "negativo"),
    ("Las sillas del auditorio eran incómodas.", "negativo"),
    ("El aeropuerto estaba desorganizado y con largas filas.", "negativo"),
    ("El desayuno incluido era de mala calidad.", "negativo"),
    ("La película estaba mal editada.", "negativo"),
    ("El pedido llegó incompleto.", "negativo"),
    ("La música del local estaba demasiado alta.", "negativo"),
    ("La compra fue cancelada sin explicación.", "negativo"),
    ("El producto no coincide con la descripción.", "negativo"),
    ("La conferencia empezó con una hora de retraso.", "negativo"),
    ("No pude entrar al evento porque se sobrevendió.", "negativo"),
    ("El hotel no tenía agua caliente.", "negativo"),
    ("El envío fue más caro que el producto.", "negativo"),
    ("La llamada se cortaba todo el tiempo.", "negativo"),
    ("El parque estaba lleno de basura.", "negativo"),
    ("El dispositivo se sobrecalienta fácilmente.", "negativo"),
    ("No había personal suficiente para atender a todos.", "negativo"),
    ("El coche que alquilé tenía el tanque vacío.", "negativo"),
    ("El trámite fue lento y burocrático.", "negativo"),
    ("No pude conectarme a la red WiFi del lugar.", "negativo"),
    ("La calidad del material era muy baja.", "negativo"),
    ("El evento fue cancelado sin previo aviso.", "negativo"),
    ("El cine estaba sucio y olía mal.", "negativo"),
    ("La comida tardó demasiado en llegar.", "negativo"),
    ("La mesa estaba pegajosa y sucia.", "negativo"),
    ("El producto venía con piezas rotas.", "negativo"),
    ("El espectáculo no cumplió mis expectativas.", "negativo"),
    ("El precio era demasiado alto para lo que ofrecían.", "negativo"),
    ("El bar estaba demasiado lleno y caluroso.", "negativo"),
    ("El tour fue mal organizado.", "negativo"),
    ("La luz se fue en medio de la cena.", "negativo"),
    ("El transporte llegó tarde y sin disculpas.", "negativo"),
    ("El servicio de atención telefónica fue inútil.", "negativo"),
    ("El aroma del lugar era desagradable.", "negativo"),
    ("El micrófono falló durante la presentación.", "negativo"),
    ("La comida estaba recalentada.", "negativo"),
    ("El pedido se perdió en el envío.", "negativo"),
    ("El menú tenía muy pocas opciones.", "negativo"),
    ("El pago con tarjeta no funcionaba.", "negativo"),
    ("El cuarto de baño estaba descuidado.", "negativo"),
    ("La aplicación se bloquea al iniciar.", "negativo"),
    ("El vendedor no cumplió su palabra.", "negativo"),
    ("El lugar estaba en remodelación y no avisaron.", "negativo"),
    ("El ruido de las obras fue insoportable.", "negativo"),
    ("El pedido llegó sin el empaque adecuado.", "negativo"),
    ("El precio en caja era mayor que el anunciado.", "negativo"),
    ("El sabor del postre era artificial.", "negativo"),
    ("La fila para entrar era interminable.", "negativo"),
    ("El personal no estaba capacitado.", "negativo"),
    ("El aire estaba muy contaminado.", "negativo"),
    ("El producto llegó con moho.", "negativo"),
    ("La habitación tenía insectos.", "negativo"),
    ("El servicio en línea nunca responde.", "negativo"),
    ("El restaurante estaba cerrado sin aviso previo.", "negativo"),
    ("El concierto fue interrumpido varias veces.", "negativo"),
    ("La pantalla del televisor estaba rayada.", "negativo"),
    ("El transporte no pasó a la hora indicada.", "negativo"),
    ("La sopa estaba aguada y sin sabor.", "negativo"),
    ("El hotel estaba en una zona peligrosa.", "negativo"),
    ("El teléfono llegó sin cargador.", "negativo"),
    ("El evento no tenía suficientes asientos.", "negativo"),
    ("El equipo de sonido era de mala calidad.", "negativo"),
    ("El museo estaba en remodelación y casi no había exhibiciones.", "negativo"),
    ("El agua del grifo estaba turbia.", "negativo"),
    ("La comida tenía un sabor extraño.", "negativo"),
    ("El artículo tenía defectos de fábrica.", "negativo"),
    ("El servicio fue lento y desorganizado.", "negativo"),
    ("El lugar estaba mal iluminado.", "negativo"),
    ("El producto vino en un color equivocado.", "negativo"),
    ("El tour fue cancelado por mal clima, pero no devolvieron el dinero.", "negativo"),
    ("El estacionamiento era muy costoso.", "negativo"),
    ("El hotel no tenía internet en las habitaciones.", "negativo"),
    ("El ambiente del local era incómodo.", "negativo"),
    ("El pedido se retrasó por más de una semana.", "negativo"),
    ("El servicio de streaming se detiene constantemente.", "negativo"),
    ("El bar estaba muy sucio.", "negativo"),
]

# Guardar
df = pd.DataFrame(training_data, columns=["texto", "etiqueta"])
df.to_csv("dataset_sentimientos_100.csv", index=False, encoding="utf-8")


Guardo la información en un dataframe para facilitar su manipulación, análisis y procesamiento posterior.

### 2. Cargue de datos

In [133]:
df = pd.read_csv('dataset_sentimientos_100.csv')
df.head()

,texto,etiqueta
0,"Me encantó la película, la historia fue emocio...",positivo
1,La comida en ese lugar siempre me deja con una...,positivo
2,Gracias al equipo de soporte por resolver mi p...,positivo
3,El nuevo diseño de la aplicación es muy intuit...,positivo
4,"Disfruté muchísimo la conferencia, aprendí más...",positivo


Analizo la distribución de las clases para detectar posibles desbalances, garantizando que el modelo aprenda de manera equilibrada y evitando sesgos en la predicción.

In [134]:
df.etiqueta.value_counts()

etiqueta
positivo    100
negativo    100
Name: count, dtype: int64

### 3. Procesamiento NLP

Convierto la etiqueta POS generada por spaCy a su equivalente en WordNet, con el fin de facilitar tareas de procesamiento de lenguaje natural como lematización, análisis semántico y búsqueda de sinónimos.  
### Parámetros:
    spacy_pos (str): Etiqueta POS de spaCy (ej. 'NOUN', 'VERB', etc.)

### Retorna:
    str | None: Etiqueta POS en formato WordNet o None si no aplica.

In [135]:
def get_wordnet_pos(upos):
    # Sustantivo
    if upos.startswith('N'):
        return wn.NOUN
    # Verbo
    elif upos.startswith('V'):
        return wn.VERB
    # Adjetivo
    elif upos.startswith('J'):
        return wn.ADJ
    # Adverbio
    elif upos.startswith('R'):
        return wn.ADV
    # Etiqueta no mapeada
    else:
        return None

### 4. Score de sentimientos

Se calcula el sentimiento de un texto en español mediante un flujo que incluye traducción automática al inglés, análisis morfosintáctico con spaCy, mapeo robusto de etiquetas POS a formato WordNet y asignación de puntajes desde SentiWordNet. El proceso incorpora lematización, búsqueda con y sin POS para mayor cobertura, y detección de negaciones para ajustar los puntajes, permitiendo así una evaluación semántica más precisa y coherente.

### Flujo:
    
    Traduce el texto (es → en) usando GoogleTranslator.
    Procesa con spaCy para obtener lemas y etiquetas POS.
    Mapea POS a formato WordNet (con fallback sin POS si no hay coincidencia).
    Calcula puntajes positivos y negativos desde SentiWordNet. 
    Ajusta puntajes si hay negaciones cercanas a la palabra

### Parámetros:
    text (str): Texto de entrada en español.

### Retorna:
    str: "positivo" o "negativo".

In [136]:
def sentiment_score(text):
    # Traducir el texto de cualquier idioma al inglés (SentiWordNet en inglés)
    translated = GoogleTranslator(source='auto', target='en').translate(text)

    # Procesar el texto con spaCy para obtener tokens, lemas, POS, dependencias, etc.
    doc = nlp(translated)

    # Inicializar acumuladores de puntajes
    pos_score = neg_score = 0
    count = 0

    # Lista de palabras de negación que invierten el sentido del sentimiento
    negations = {"not", "no", "never", "n't", "without"}

    # Recorrer cada token del texto traducido
    for token in doc:
        # Ignorar palabras vacías y signos de puntuación
        if token.is_stop or token.is_punct:
            continue

        # Obtener lema en minúsculas
        lemma = token.lemma_.lower()

        # Obtener etiqueta POS adaptada a formato de WordNet (tag_ o pos_ como respaldo)
        wn_pos = get_wordnet_pos(token.tag_) or get_wordnet_pos(token.pos_)

        # Buscar synsets (significados) en WordNet con POS
        synsets = wn.synsets(lemma, pos=wn_pos) if wn_pos else []

        # Si no encuentra con POS, buscar sin especificar POS (fallback)
        if not synsets:
            synsets = wn.synsets(lemma)

        # Si encuentra al menos un synset
        if synsets:
            try:
                # Obtener el synset en SentiWordNet y sus puntajes positivo/negativo
                swn_synset = swn.senti_synset(synsets[0].name())
                token_pos = swn_synset.pos_score()
                token_neg = swn_synset.neg_score()

                # Ajustar puntajes si hay una palabra de negación a la izquierda del token
                if any(t.text.lower() in negations for t in token.lefts):
                    token_pos, token_neg = token_neg, token_pos  # Invertir puntajes

                # Acumular puntajes en total
                pos_score += token_pos
                neg_score += token_neg
                count += 1

            except:
                # Si hay error (por ejemplo, no existe en SentiWordNet) se ignora el token
                continue

    # Normalizar puntajes para obtener promedios
    if count > 0:
        pos_score /= count
        neg_score /= count

    # Clasificar: solo retorna positivo o negativo
    return "positivo" if pos_score >= neg_score else "negativo"

Se aplica la función sentiment_score a cada texto del DataFrame, calculando la polaridad del sentimiento y almacenando el resultado en la columna 'prediccion' para su posterior análisis o evaluación del modelo.

In [137]:
df["prediccion"] = df["texto"].apply(sentiment_score)

### 5. Evaluación de desempeño del modelo

In [138]:
# Etiquetas verdaderas y predicciones
y_true = df.etiqueta.values
y_pred = df.prediccion.values

# Calcular accuracy
acc = accuracy_score(y_true, y_pred)

# Matriz de confusión para las tres clases
cm = confusion_matrix(y_true, y_pred, labels=["positivo", "negativo"])

# Reporte de clasificación con precision, recall y f1-score
cr = classification_report(y_true, y_pred, labels=["positivo", "negativo"])

# Mostrar resultados
print(f"Accuracy:\n{acc}\n")
print(f"Classification Report:\n{cr}")
print(f"Confusion Matrix:\n{cm}")

Accuracy:
0.705

Classification Report:
              precision    recall  f1-score   support

    positivo       0.67      0.82      0.74       100
    negativo       0.77      0.59      0.67       100

    accuracy                           0.70       200
   macro avg       0.72      0.70      0.70       200
weighted avg       0.72      0.70      0.70       200

Confusion Matrix:
[[82 18]
 [41 59]]


### 6. Resultados

In [139]:
print(df)

                                                 texto  etiqueta prediccion
0    Me encantó la película, la historia fue emocio...  positivo   positivo
1    La comida en ese lugar siempre me deja con una...  positivo   positivo
2    Gracias al equipo de soporte por resolver mi p...  positivo   negativo
3    El nuevo diseño de la aplicación es muy intuit...  positivo   positivo
4    Disfruté muchísimo la conferencia, aprendí más...  positivo   positivo
..                                                 ...       ...        ...
195    El hotel no tenía internet en las habitaciones.  negativo   positivo
196                El ambiente del local era incómodo.  negativo   negativo
197        El pedido se retrasó por más de una semana.  negativo   positivo
198  El servicio de streaming se detiene constantem...  negativo   negativo
199                           El bar estaba muy sucio.  negativo   negativo

[200 rows x 3 columns]


In [140]:
print(df[df["etiqueta"] != df["prediccion"]])

                                                 texto  etiqueta prediccion
2    Gracias al equipo de soporte por resolver mi p...  positivo   negativo
11   Este libro me atrapó desde la primera página, ...  positivo   negativo
12   La música en vivo fue una experiencia que no o...  positivo   negativo
17   La presentación estuvo impecable, bien estruct...  positivo   negativo
25   El software funciona a la perfección, sin erro...  positivo   negativo
30   El pastel que preparaste estaba espectacular, ...  positivo   negativo
31   La atención de los meseros fue impecable toda ...  positivo   negativo
37   El servicio técnico resolvió el problema en un...  positivo   negativo
46    La charla motivacional me dejó lleno de energía.  positivo   negativo
52            El trato que recibí fue cálido y humano.  positivo   negativo
53   Las clases de yoga me ayudaron a relajarme muc...  positivo   negativo
66                 La serie que vi anoche es adictiva.  positivo   negativo
72          

141 textos fueron etiquetados correctamente y 59 fueron mal clasificados, obteniendo un accuracy del 70%, lo que indica que el modelo tiene un desempeño aceptable pero con margen de mejora, especialmente en la reducción de falsos positivos y falsos negativos.

##### Los casos donde no coincide la etiqueta con la predicción parece que no cuentan con alguna negación en el caso de los comentarios negativos. Por otro lado, parece que al modelo le cuesta más clasificar los casos positivos cuando son comentarios muy largos. 

### 7. Conclusiones

#### Eficacia del flujo de análisis
- El pipeline diseñado —traducción automática al inglés, análisis POS con spaCy, mapeo a WordNet y cálculo de puntajes con SentiWordNet— permitió evaluar el sentimiento de textos en español sin necesidad de un corpus entrenado específicamente en este idioma.

#### Rendimiento del modelo
- Se clasificaron correctamente 141 textos y 59 fueron mal clasificados, alcanzando un accuracy del 70%. Esto refleja un rendimiento moderado, útil como aproximación inicial, pero insuficiente para aplicaciones que requieran alta precisión.

#### Limitaciones observadas

- La traducción automática puede introducir cambios léxicos y semánticos que afectan la identificación de synsets y, por ende, los puntajes de sentimiento.
- SentiWordNet presenta cobertura limitada para ciertas palabras poco comunes, coloquialismos o expresiones idiomáticas.
- La estrategia de selección de synsets más relevantes aún es básica y podría mejorarse incorporando desambiguación semántica más robusta.

#### Áreas de mejora

- Implementar un modelo de sentiment analysis entrenado directamente en español para reducir el sesgo de la traducción.
- Mejorar la desambiguación de sentidos de palabra para elegir synsets más representativos del contexto.
- Incorporar manejo más avanzado de negaciones, intensificadores y sarcasmo.

#### Valor práctico
- A pesar de sus limitaciones, el método ofrece una solución funcional y relativamente sencilla para análisis de sentimiento en español, siendo un buen punto de partida para pruebas rápidas antes de migrar a soluciones más sofisticadas basadas en aprendizaje profundo.